In [1]:
from config import DATA_DIR
from utils import sample_series, save_df, read_df
import os
import pandas as pd
import nltk
import re
from unidecode import unidecode

Abrindo os dados originais parseados

In [2]:
df: pd.DataFrame = read_df('dados_orginais_contribuicoes.csv')

In [3]:
df.sample(3)

,id,canal,texto,temas
2061,2802,Participe+,oficinas no Parque do Chuvisco. A subprefeitur...,['Cultura']
1419,2154,Participe+,Implantação do Parque Arqueológico Cavas de Ou...,['Meio Ambiente']
46,47,Audiência Regional - V Maria V Guilherme,"Um boa noite a todos, quero saudar todos os pa...","['Urbanismo e Licenciamento', 'Habitação']"


Limpezas simples do texto -- tirar maiusculas etc.

In [4]:
df.rename({'texto' : 'texto_original'}, axis=1, inplace=True)

In [5]:
df['texto_limpo'] = df['texto_original'].str.lower().str.strip('/n')


In [6]:
print(sample_series(df['texto_limpo']))

meta 37 - iniciativa j - revisar - incluir a implantação de faixa elevada de pedestre ao longo da calçada verde da rua manuel dos reis araújo no jardim marajoara para melhoria da segurança dos munícipes que praticam caminhadas e diversas outras atividades físicas e de lazer no local.


Baixando as stopwords para limpar os textos

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/h-pgy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [9]:
stopwords[:3]

['a', 'à', 'ao']

In [10]:
type(stopwords)

list

In [11]:
pdm_stopwords: list[str] = ['programa de metas',
                            'pdm',
                            'audiência',
                            'audiencia',
                            'boa noite',
                            'bom dia',
                            'boa tarde',
                            'meta',
                            'olá',
                            'iniciativa'
                        ]

In [12]:
stopwords.extend(pdm_stopwords)

stopwords = set(stopwords)

tokenizando o texto

In [13]:
from nltk.tokenize import word_tokenize

In [14]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/h-pgy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [15]:
df['tokens'] = df['texto_limpo'].apply(word_tokenize)

In [16]:
print(sample_series(df['tokens']))

['meta', '37', '-', 'iniciativa', 'j', '-', 'revisar', '-', 'incluir', 'a', 'implantação', 'de', 'faixa', 'elevada', 'de', 'pedestre', 'ao', 'longo', 'da', 'calçada', 'verde', 'da', 'rua', 'manuel', 'dos', 'reis', 'araújo', 'no', 'jardim', 'marajoara', 'para', 'melhoria', 'da', 'segurança', 'dos', 'munícipes', 'que', 'praticam', 'caminhadas', 'e', 'diversas', 'outras', 'atividades', 'físicas', 'e', 'de', 'lazer', 'no', 'local', '.']


In [17]:
def remove_stopwords(tokens:str, stopwords:set[str]=stopwords)->list[str]:

    filtered: list[str]=[token for token in tokens
                         if token not in stopwords]
    
    return filtered





In [18]:
df['tokens_clean'] = df['tokens'].apply(remove_stopwords)

In [19]:
print(sample_series(df['tokens_clean']))

['37', '-', 'j', '-', 'revisar', '-', 'incluir', 'implantação', 'faixa', 'elevada', 'pedestre', 'longo', 'calçada', 'verde', 'rua', 'manuel', 'reis', 'araújo', 'jardim', 'marajoara', 'melhoria', 'segurança', 'munícipes', 'praticam', 'caminhadas', 'diversas', 'outras', 'atividades', 'físicas', 'lazer', 'local', '.']


Remover caracteres especiais e pontuacao

In [20]:
def remover_char_especiais(tokens:list[str])->list[str]:

    #tudo o que não são caracteres alphanumericos
    patt = r'[^\w]'
    limpo:list[str] = []
    for token in tokens:
        token = re.sub(patt, '', token)
        if token:
            limpo.append(token)
    return limpo

In [21]:
remover_char_especiais(['ontem', '123', '@'])

['ontem', '123']

In [22]:
df['tokens_clean'] = df['tokens_clean'].apply(remover_char_especiais)

In [23]:
print(sample_series(df['tokens_clean']))

['37', 'j', 'revisar', 'incluir', 'implantação', 'faixa', 'elevada', 'pedestre', 'longo', 'calçada', 'verde', 'rua', 'manuel', 'reis', 'araújo', 'jardim', 'marajoara', 'melhoria', 'segurança', 'munícipes', 'praticam', 'caminhadas', 'diversas', 'outras', 'atividades', 'físicas', 'lazer', 'local']


Rmover os números - são referência às metas, podem enviesar o modelo.

In [24]:
'214'.isdigit()

True

In [25]:
def remove_numbers(tokens:list[str])->list[str]:

    cleaned_tokens: list[str] = []

    for token in tokens:
        while token.startswith('0'):
            token=token.lstrip('0')
        
        if not token.isdigit():
            cleaned_tokens.append(token)

    return cleaned_tokens

In [26]:
df['tokens_clean'] = df['tokens_clean'].apply(remove_numbers)

In [27]:
print(sample_series(df['tokens_clean']))

['j', 'revisar', 'incluir', 'implantação', 'faixa', 'elevada', 'pedestre', 'longo', 'calçada', 'verde', 'rua', 'manuel', 'reis', 'araújo', 'jardim', 'marajoara', 'melhoria', 'segurança', 'munícipes', 'praticam', 'caminhadas', 'diversas', 'outras', 'atividades', 'físicas', 'lazer', 'local']


remover acentos

In [28]:
def remover_acentos(tokens:list[str])->list[str]:

    return [unidecode(token) for token in tokens]

In [29]:
df['tokens_clean'] = df['tokens_clean'].apply(remover_acentos)

In [30]:
print(sample_series(df['tokens_clean']))

['j', 'revisar', 'incluir', 'implantacao', 'faixa', 'elevada', 'pedestre', 'longo', 'calcada', 'verde', 'rua', 'manuel', 'reis', 'araujo', 'jardim', 'marajoara', 'melhoria', 'seguranca', 'municipes', 'praticam', 'caminhadas', 'diversas', 'outras', 'atividades', 'fisicas', 'lazer', 'local']


In [31]:
save_df(df, 'dados_tokenizados.csv')

'../data/dados_tokenizados.csv'